# AlpaCare Medical Assistant - Fine-tuning Notebook
## Solar Industries India Ltd - Internship Assessment

This notebook fine-tunes a pre-trained model on medical instructions using LoRA.

## 1. Install Dependencies

In [ ]:
!pip install -q torch transformers datasets peft accelerate bitsandbytes einops trl scipy safetensors

## 2. Import Libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import os
from data_loader import load_medical_dataset, preprocess_function

print("Libraries imported successfully!")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 3. Model & Tokenizer Setup

In [ ]:
# Model choice: Microsoft DialoGPT-medium - Small, fast, good for conversations
model_name = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model and tokenizer loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")

## 4. LoRA Configuration

In [ ]:
lora_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj", "c_fc"]  # GPT-2 attention modules
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("LoRA configuration applied!")

## 5. Load and Preprocess Data

In [ ]:
# Load dataset
dataset_splits = load_medical_dataset()

# Use smaller subset for demo (full training in actual scenario)
train_dataset = dataset_splits['train'].select(range(1000))  # First 1000 samples for demo
eval_dataset = dataset_splits['validation'].select(range(200))  # 200 samples for eval

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

# Preprocess data
tokenized_train = train_dataset.map(
    lambda x: preprocess_function(x, tokenizer),
    batched=True
)

tokenized_eval = eval_dataset.map(
    lambda x: preprocess_function(x, tokenizer),
    batched=True
)

print("Data preprocessing completed!")

## 6. Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./alpacare-medical-assistant",
    overwrite_output_dir=True,
    num_train_epochs=1,  # One epoch as required
    per_device_train_batch_size=4,  # Small batch for Colab memory
    per_device_eval_batch_size=4,
    warmup_steps=100,
    logging_steps=50,
    eval_steps=100,
    save_steps=200,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=None,  # Disable wandb/etc
    fp16=True,  # Enable mixed precision
)

print("Training arguments configured!")

## 7. Initialize Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

print("Trainer initialized! Ready for training.")

## 8. Start Training

In [ ]:
print("Starting training...")
trainer.train()
print("Training completed!")

## 9. Save LoRA Adapter

In [ ]:
# Save the LoRA adapter
adapter_path = "./alpacare-lora-adapter"
trainer.save_model(adapter_path)

# Also save via PeftModel to be safe
model.save_pretrained(adapter_path)

print(f"LoRA adapter saved to: {adapter_path}")

# Create zip file for download
!zip -r alpacare-lora-adapter.zip {adapter_path}

print("Adapter zip file created! You can download it from Colab files section.")

## ✅ Training Complete!

**Next Steps:**
1. Download `alpacare-lora-adapter.zip` from Colab files
2. Use the inference notebook to test the model
3. Upload adapter to your GitHub repository